In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
dtype = torch.FloatTensor

In [ ]:
sentence = ["가 을 은 독 서 의 계 절 \t B I I B I I B I", "새 로 운 해 가 밝 았 다 \t B I I B I B I I",
            "내 가 쿠 키 를 구 웠 어 요 \t B I B I I B I I I", "당 신 은 나 를 본 적 이 있 나 요 ? \t B I I B I B B I B I I I"]

In [ ]:
datas = []
eumjeol_sequence = [] # 음절 집합
correct_labels = [] # 정답 집합

for sen in sentence:
    pieces = sen.strip().split("\t")
    sequence, label = pieces[0].strip().split(), pieces[1].strip().split()

    eumjeol_sequence.append(sequence)
    correct_labels.append(label)

    datas.append((sequence, label))

In [ ]:
word = []
max_len = 0
for sequence, label in datas:
    if len(sequence) > max_len:
        max_len = len(sequence)
    for sen in sequence:
        if sen not in word:
            word.append(sen)

    word_dict = {w:i for i, w in enumerate(word)}
    number_dict = {i:w for i, w in enumerate(word)}

In [ ]:
tag2idx = {"PAD_ID":0, "B":1, "I":2}
idx2tag = {0:"PAD_ID", 1:"B", 2:"I"}

n_size = len(sentence) # 문장 개수
n_class = len(tag2idx) # 띄어쓰기 여부
vocab_size = len(word_dict)
n_hidden = 64
n_embedding = 50

In [ ]:
def make_batch(datas):
    input_batch = []
    target_batch = []

    for sentence, label in datas:
        input = []
        target = []
        for n, sen in enumerate(sentence):
            input.append(word_dict[sen])
            target.append(tag2idx[label[n]])

        input = input + [0] * (max_len - len(input))
        target = target + [tag2idx["PAD_ID"]] * (max_len - len(target))
        input_batch.append(input)
        target_batch.append(target)
    return Variable(torch.LongTensor(input_batch)), Variable(torch.LongTensor(target_batch))

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()

        self.embed = nn.Embedding(vocab_size, n_embedding)
        self.lstm = nn.LSTM(input_size = n_embedding, hidden_size = n_hidden, bidirectional = True, batch_first = True) 
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X): 
        input = self.embed(X)
        
        hidden_state = Variable(torch.zeros(1*2, len(X), n_hidden)) # [num_layers(=1)* num_directions(=2), batch_size, n_hidden)
        cell_state = Variable(torch.zeros(1*2, len(X), n_hidden))	# [num_layeis(=1)* num_directions(=2), batch_size, n_hidden)

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        predictions = self.fc(outputs) 
        return predictions.permute(0, 2, 1)



In [ ]:
input_batch, target_batch = make_batch(datas) 
model = BiLSTM()

criterion = nn.CrossEntropyLoss(ignore_index=tag2idx["PAD_ID"]) 
optimizer = optim.Adam(model.parameters(), lr = 0.001) 

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch) 
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d'% (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        
    loss.backward()
    optimizer.step()

predict = model(input_batch).data.max(1, keepdim = True)[1]

In [ ]:
model(input_batch)

In [ ]:
predict_labels = predict.numpy() 
print(predict_labels) 
for i in range(len(eumjeol_sequence)): 
    correct_sentence = ''
    predict_sentence = ''
    for j in range(en(eumjeol_sequence[i])): 
        if (j!= 0) & (correct_labels[i][j] == 'B'): 
            correct_sentence += " " 
        correct_sentence += eumjeol_sequence[i][j]

        if (j != 0) & (predict_label[i][0][j] == 1): 
            predict_sentence += " " 
            predict_sentence += eumjeol_sequence[i][j] 
        
    print('정답 문장:' + correct_sentence) 
    print('출력 문장:' predict_sentence) 

In [ ]:
self.embed = nn.Embedding(vocab_size, n_embedding) It> selfIstm = nn.LSTM(input_size=n_embedding, hidden_size=n_hidden, bidirectional=True, batch_first=True) self.fc = nn.Linear(n_hidden * 2, n_class) 
def forward(self, X): input = self.embed(X) 
hidden_state = Variable(torch.zeros(1*2, len(X), n_hidden))
cell_state = vanabie(torcn.zeros(iwz, len(X), n_nicicien))
outputs, (_, J = selfistm(input, (hidden_state, cell_state)) 
predictions = self.fc(outputs) return predictions.permute(0, 2, 1) 